<a href="https://colab.research.google.com/github/eduardoguimaraes90/projeto-final-sghss-backend/blob/main/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalações
!pip install fastapi uvicorn pyngrok nest-asyncio

In [2]:
# Escreve o arquivo main.py com a API completa
%%writefile main.py

from fastapi import FastAPI, Depends, HTTPException, status
from pydantic import BaseModel
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime
from typing import List

# --- Configuração do Banco de Dados (SQLite) ---
SQLALCHEMY_DATABASE_URL = "sqlite:///./sghss_vidaplus.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# --- Modelos do Banco de Dados (Tabelas) ---

class PacienteDB(Base):
    __tablename__ = "pacientes"
    id = Column(Integer, primary_key=True, index=True)
    nome = Column(String, index=True)
    cpf = Column(String, unique=True, index=True)

class ConsultaDB(Base):
    __tablename__ = "consultas"
    id = Column(Integer, primary_key=True, index=True)
    paciente_id = Column(Integer, index=True)
    profissional_id = Column(Integer, index=True)
    data_hora = Column(DateTime, index=True)
    status = Column(String, default="Confirmado")

# Cria TODAS as tabelas no banco de dados
Base.metadata.create_all(bind=engine)

# --- Schemas (Modelos de Dados da API) ---

# Schemas para Paciente
class PacienteCreate(BaseModel):
    nome: str
    cpf: str

class PacienteRead(PacienteCreate):
    id: int
    class Config:
        orm_mode = True

# Schemas para Agendamento
class AgendamentoCreate(BaseModel):
    paciente_id: int
    profissional_id: int
    data_hora: datetime

class AgendamentoRead(AgendamentoCreate):
    id: int
    status: str
    class Config:
        orm_mode = True

# --- Lógica da API ---
app = FastAPI(title="SGHSS VidaPlus - Protótipo Completo")

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# --- ENDPOINTS DE PACIENTES ---

@app.post("/pacientes/", response_model=PacienteRead, status_code=status.HTTP_201_CREATED)
def criar_paciente(paciente: PacienteCreate, db: Session = Depends(get_db)):
    db_paciente_existente = db.query(PacienteDB).filter(PacienteDB.cpf == paciente.cpf).first()
    if db_paciente_existente:
        raise HTTPException(status_code=409, detail="Paciente com este CPF já cadastrado.")

    db_paciente = PacienteDB(**paciente.dict())
    db.add(db_paciente)
    db.commit()
    db.refresh(db_paciente)
    return db_paciente

@app.get("/pacientes/", response_model=List[PacienteRead])
def listar_pacientes(db: Session = Depends(get_db)):
    pacientes = db.query(PacienteDB).all()
    return pacientes


# --- ENDPOINTS DE AGENDAMENTOS ---

@app.post("/agendamentos/", response_model=AgendamentoRead, status_code=status.HTTP_201_CREATED)
def criar_agendamento(agendamento: AgendamentoCreate, db: Session = Depends(get_db)):
    # Validação: Verifica se o paciente existe
    paciente = db.query(PacienteDB).filter(PacienteDB.id == agendamento.paciente_id).first()
    if not paciente:
        raise HTTPException(status_code=404, detail=f"Paciente com ID {agendamento.paciente_id} não encontrado.")

    # Validação de conflito de horário
    conflito = db.query(ConsultaDB).filter(
        ConsultaDB.profissional_id == agendamento.profissional_id,
        ConsultaDB.data_hora == agendamento.data_hora
    ).first()
    if conflito:
        raise HTTPException(status_code=409, detail="Horário indisponível para este profissional.")

    db_agendamento = ConsultaDB(**agendamento.dict())
    db.add(db_agendamento)
    db.commit()
    db.refresh(db_agendamento)
    return db_agendamento

Overwriting main.py


In [3]:
# Célula 3: Iniciar o Servidor e Expor com ngrok (VERSÃO SEGURA PARA GITHUB)
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from google.colab import userdata # Importa a biblioteca para ler segredos

# --- Autenticação Segura do ngrok ---
# Lê o token a partir dos "Secrets" do Colab. O código pode ser público!
try:
    NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
except userdata.SecretNotFoundError:
    print("ERRO: Segredo 'NGROK_AUTHTOKEN' não encontrado.")
    print("Por favor, adicione seu token do ngrok nos Segredos (ícone de chave 🔑) para continuar.")
# ---------------------------------------------

nest_asyncio.apply()
port = 8000
ngrok.kill()
public_url = ngrok.connect(port, "http")
print(f"✅ API está rodando e acessível em: {public_url}")

# Inicia o servidor uvicorn
uvicorn.run("main:app", host="0.0.0.0", port=port, log_level="info")

✅ API está rodando e acessível em: NgrokTunnel: "https://8742-35-230-122-24.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [81330]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     131.255.178.148:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     131.255.178.148:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     131.255.178.148:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     131.255.178.148:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     131.255.178.148:0 - "GET /pacientes/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [81330]
